# Regression and Classification with the Ames Housing Data

---

## Premise: 
I have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

## Data:
The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt

## Objective:
1. Develop an algorithm to estimate the value of the residential homes based on fixed characteristics (those that are not considered easy to renovate).
2. Identify characteristics of homes that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Use the information to buy homes that are likely to sell for more than the initial purchase.

## Process:
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize and evaluate the model. (How does it perform and what are the best estimates of price?)

## Citations:
- http://www.remodeling.hw.net/cost-vs-value/2010/west-north-central/des-moines-ia/
- http://cdnassets.hw.net/b6/3d/047accdd4174a4965051631d7900/cvv-2010-2011-professional-desmoinesia.pdf

In [1]:
!pip install display --quiet

In [2]:
import sys
import numpy as np, pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt, seaborn as sns

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### Load and Inspect the Data

In [3]:
from os import chdir

In [4]:
pwd

'/home/jovyan/DSI/DSI_Plus_1_Curriculum/project-three/project-three/ipynb'

In [5]:
chdir('../')

In [6]:
from library import functions as fy

In [7]:
clean_house_df = pd.read_pickle('./assets/clean_house_df.p')

In [8]:
clean_house_df.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
680,681,120,RL,50.0,8012,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,5,2010,WD,Normal,143000
545,546,50,RL,70.0,13837,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,2,2006,WD,Normal,229000
924,925,20,RL,79.0,10240,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,5,2006,WD,Normal,207500
1304,1305,160,RM,32.0,3363,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2006,WD,Normal,130000
1032,1033,60,RL,70.0,14541,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,11,2006,WD,Abnorml,310000


**The size of my dataset is 3.2 MB as shown below:**

In [9]:
sys.getsizeof(clean_house_df)/1000000

3.258239

### Understanding the Objective

In order to meet the 1st objective (which is to build a model using the fixed (not easily renovatable) features to predict the value of a residential home (`SalePrice`), I did some research online to help me roughly determine what features are considered difficult to renovate. My research led me to the following website and pdf:
    - http://www.remodeling.hw.net/cost-vs-value/2010/west-north-central/des-moines-ia/
    - http://cdnassets.hw.net/b6/3d/047accdd4174a4965051631d7900/cvv-2010-2011-professional-desmoinesia.pdf
Together with the data dictionary, the following features were categorized into the following 3 categories:
- features that cannot be changed
- features that are difficult to fix
- features that are less difficult to fix

In [10]:
clean_house_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageArea', 'GarageQual', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', '

In [11]:
clean_house_df.drop('MiscVal', axis=1, inplace=True)

In [12]:
len(clean_house_df.columns)

69

In [13]:
cannot_change = ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape', 'LandContour',\
                 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', \
                 'HouseStyle','YearBuilt', 'YearRemodAdd', 'Foundation', 'BsmtQual', 'BsmtFinSF1', 'BsmtFinSF2', \
                 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageType', 'GarageYrBlt',\
                 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',\
                 'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice']

In [14]:
lesshardtofix = ['BsmtFinSF1', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'LowQualFinSF',  'KitchenQual',\
                'TotRmsAbvGrd', 'GarageFinish', 'GarageQual', 'PavedDrive',]

In [15]:
import pickle

In [16]:
pickle.dump(lesshardtofix, open('lesshardtofix.p', 'wb'))

In [17]:
hardtofix = ['OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl','Exterior1st', 'ExterQual','ExterCond',\
             'MasVnrType', 'MasVnrArea', 'BsmtExposure', 'BsmtFullBath', 'BsmtHalfBath', \
             'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Functional', 'Fireplaces', 'GarageArea',]

In [18]:
pickle.dump(hardtofix, open('hardtofix.p', 'wb'))

In [19]:
display(len(cannot_change))
display(len(lesshardtofix))
display(len(hardtofix))

40

11

19

### Examine the non-renovatable features of a home to estimate home value

In [20]:
cannot_change_df = clean_house_df[cannot_change] 
cannot_change_df.shape

(1385, 40)

In [21]:
cannot_change_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'YearBuilt', 'YearRemodAdd', 'Foundation', 'BsmtQual', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'GarageType', 'GarageYrBlt', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold',
       'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

#### Define the target `SalePrice` and the predictive features

In [22]:
y = cannot_change_df['SalePrice']

In [23]:
X = clean_house_df[cannot_change].drop(['SalePrice', 'Id'], axis=1)

In [24]:
display(X.shape)
display(y.shape)

(1385, 38)

(1385,)

In [25]:
X['YrSold'].value_counts()

2009    321
2007    313
2006    299
2008    288
2010    164
Name: YrSold, dtype: int64

#### Do a manual train/test/split where if the Yr Sold is 2010, then it is the train set, otherwise if the year sold is 2006-2009, then it is in the train set.

In [26]:
X_test = X[X['YrSold'] == 2010]
X_train = X[X['YrSold'] != 2010]

In [27]:
display(X_test.shape)
display(X_train.shape)

(164, 38)

(1221, 38)

In [28]:
y_test = cannot_change_df[cannot_change_df['YrSold'] == 2010]['SalePrice']
y_test.shape

(164,)

In [29]:
y_train = cannot_change_df[cannot_change_df['YrSold'] != 2010]['SalePrice']
y_train.shape

(1221,)

In [30]:
pickle.dump(y_test, open('y_test.p', 'wb'))
pickle.dump(y_train, open('y_train.p', 'wb'))

There are some features that are categorial, while there are others that are numeric. In order to enhance the performance of a regression model, I will convert the categorial fetaures to numeric by using pandas `get_dummies`. Below, I separate out the cateogiral from the numeric feature based on the data dictionary.

In [31]:
display(X.columns)
display(len(X.columns))

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'YearBuilt', 'YearRemodAdd', 'Foundation', 'BsmtQual', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'GarageType', 'GarageYrBlt', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold',
       'YrSold', 'SaleType', 'SaleCondition'],
      dtype='object')

38

In [32]:
fixed_categorical = ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', \
                 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2','BldgType', 'HouseStyle', 'Foundation',\
                'BsmtQual', 'GarageType', 'SaleType', 'SaleCondition']
display(len(fixed_categorical))

18

In [33]:
fixed_numeric = [column for column in X.columns if column not in fixed_categorical]
display(len(fixed_numeric))

20

In [34]:
X['GarageYrBlt'].dtype

dtype('O')

In [35]:
X['GarageYrBlt'].replace('NA', value=0, inplace=True)

In [36]:
X_dummies_df = pd.get_dummies(data = X, columns = fixed_categorical)

In [37]:
display(X_dummies_df.shape)
display(X_dummies_df.head())

(1385, 146)

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,2003,2003,706,0,150,856,856,854,...,0,0,0,1,0,0,0,0,1,0
1,80.0,9600,1976,1976,978,0,284,1262,1262,0,...,0,0,0,1,0,0,0,0,1,0
2,68.0,11250,2001,2002,486,0,434,920,920,866,...,0,0,0,1,0,0,0,0,1,0
3,60.0,9550,1915,1970,216,0,540,756,961,756,...,0,0,0,1,1,0,0,0,0,0
4,84.0,14260,2000,2000,655,0,490,1145,1145,1053,...,0,0,0,1,0,0,0,0,1,0


In [38]:
X_test = X_dummies_df[X_dummies_df['YrSold'] == 2010]
display(X_test.shape)
X_train = X_dummies_df[X_dummies_df['YrSold'] != 2010]
display(X_train.shape)

(164, 146)

(1221, 146)

In [39]:
X_test.to_pickle('./assets/X_test.p')

In [40]:
X_train.to_pickle('./assets/X_train.p')